<a href="https://colab.research.google.com/github/GeorgeFane/backtesting/blob/main/Backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get Feb 1st's top premarket gainers

In [38]:
import pandas as pd
df = pd.read_html('table.html')[0]
df

,0,1,2,3,4
0,VIE,"Viela Bio, Inc.",$52.65,17.97,+51.82%
1,API,"Agora, Inc.",$72.29,15.80,+27.97%
2,BCRX,"BioCryst Pharmaceuticals, Inc.",$10.78,2.26,+26.53%
3,ARQT,"Arcutis Biotherapeutics, Inc.",$33.26,5.97,+21.88%
4,GRTS,"Gritstone Oncology, Inc.",$21.90,3.80,+20.99%
5,AMTX,"Aemetis, Inc",$7.96,1.19,+17.58%
6,NKTR,Nektar Therapeutics,$23.10,3.40,+17.26%
7,LOTZW,"CarLotz, Inc.",$2.78,0.38,+15.83%
8,EH,EHang Holdings Limited,$76.36,10.00,+15.07%
9,ADMS,"Adamas Pharmaceuticals, Inc.",$6.78,0.87,+14.72%


In [42]:
symbols = list(df[0])
symbols

['VIE', 'API', 'BCRX', 'ARQT', 'GRTS', 'AMTX', 'NKTR', 'LOTZW', 'EH', 'ADMS']

Get price data for top movers

In [58]:
import yfinance as yf

yf.Ticker(
    'vie'
).history(
    period='1d',
    interval='2m',
).Open

Datetime
2021-02-01 09:30:00-05:00    52.709999
2021-02-01 09:32:00-05:00    52.799999
2021-02-01 09:34:00-05:00    53.000000
2021-02-01 09:36:00-05:00    53.080002
2021-02-01 09:38:00-05:00    53.055000
                               ...    
2021-02-01 15:50:00-05:00    52.889999
2021-02-01 15:52:00-05:00    52.884998
2021-02-01 15:54:00-05:00    52.884998
2021-02-01 15:56:00-05:00    52.889999
2021-02-01 15:58:00-05:00    52.868198
Name: Open, Length: 195, dtype: float64

In [59]:
prices = [
    list(yf.Ticker(
        symbol
    ).history(
        period='1d',
        interval='2m',
    ).Open) for symbol in symbols
]
df = pd.DataFrame(prices)
df

- LOTZW: No data found for this date range, symbol may be delisted


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194
0,52.709999,52.799999,53.000000,53.080002,53.055000,52.950001,52.889999,52.810001,52.799999,52.759998,52.970001,53.000000,53.000000,52.980000,52.965000,52.950001,52.959999,52.930000,52.939999,52.959999,52.930000,52.939999,52.959999,52.959999,52.939999,52.889999,52.919998,52.950001,52.970001,52.930000,52.959999,52.965000,52.959999,52.919998,52.902500,52.900002,52.950001,52.930000,52.930000,52.939999,...,52.924999,52.924999,52.924999,52.930000,52.924999,52.919998,52.924999,52.924999,52.924999,52.900002,52.900002,52.904999,52.904999,52.904999,52.910000,52.904999,52.904999,52.910000,52.895000,52.900002,52.889999,52.900002,52.900002,52.900002,52.895000,52.895000,52.889999,52.889999,52.880001,52.889999,52.889999,52.889999,52.895000,52.880001,52.889999,52.889999,52.884998,52.884998,52.889999,52.868198
1,72.080002,73.846603,72.820000,72.500000,72.589996,73.294998,73.000000,74.849998,79.260002,80.000000,79.510002,78.443497,78.839996,79.610001,79.250000,79.019997,78.800003,79.381798,79.356003,79.264397,79.180000,79.489998,79.673897,79.584999,78.760002,76.029999,76.160004,76.180000,75.800003,73.730003,75.120003,74.500000,74.690002,72.797501,72.660103,72.739998,72.714996,72.524597,72.330002,71.150101,...,72.269997,73.019997,72.750000,73.339996,73.230003,73.300003,73.849998,74.370003,74.279999,73.880402,73.800003,74.500000,74.720001,74.190002,74.300003,74.485397,74.199997,74.269997,74.699997,74.370003,74.959999,74.820000,74.639999,74.709999,75.599899,75.019997,75.294998,75.489998,75.102699,74.760002,74.540001,74.000000,73.769203,73.739998,73.779999,73.690002,73.199997,72.620003,73.300003,73.779999
2,10.750000,10.737600,10.460000,10.448100,10.500000,10.500000,10.650000,10.560100,10.470000,10.345000,10.312500,10.400000,10.345000,10.430000,10.526200,10.620000,10.940000,10.980000,11.239000,11.460000,11.315000,11.445000,11.140100,10.910000,10.930000,11.068500,10.930000,11.020000,11.090000,11.110000,11.105000,11.260000,11.120000,10.990000,10.960000,10.890000,10.935000,10.965000,10.990000,11.000000,...,11.800000,11.815000,11.750500,11.685000,11.565000,11.628500,11.680000,11.649900,11.599400,11.490000,11.460000,11.450000,11.505000,11.529900,11.585000,11.590000,11.690000,11.789000,11.740100,11.770000,11.735000,11.680100,11.710000,11.735000,11.735000,11.745000,11.745000,11.747800,11.880000,11.899800,11.945000,11.950000,11.892900,11.945000,11.919900,11.900000,11.930000,11.905000,11.875000,11.835000
3,33.250000,33.220001,33.799999,34.139999,34.549999,34.400299,34.459999,35.000000,35.000000,36.459999,35.365002,35.250000,35.250000,35.189999,35.000000,35.130001,35.009998,35.000000,35.029999,34.919998,34.939999,34.860001,34.950001,35.027699,34.990002,34.990002,35.000000,34.810101,35.000000,34.959999,34.900002,34.950001,33.990002,33.439999,33.020000,33.060001,33.889999,34.000000,34.419998,34.250000,...,35.075001,35.023800,35.130001,35.099998,35.099998,35.064999,35.064999,35.009998,34.924999,35.040001,35.130001,35.130001,35.169998,35.180000,35.165001,35.150002,35.049999,35.130001,35.139999,35.180000,35.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21.900000,21.500000,21.250000,21.000000,21.398199,21.020100,21.049999,20.570000,20.500000,20.228201,19.875000,19.920000,20.000000,19.650000,20.080000,19.860100,19.850100,19.693199,19.980000,19.969999,19.639999,19.748800,19.766001,19.882900,20.000000,20.187700,20.120001,19.815001,19.660000,19.863199,19.840000,19.700001,19.490000,19.549900,19.420000,19.400000,19.305000,19.400000,19.434999,19.450001,...,17.299999,17.450001,17.580000,17.599899,17.590000,17.580000,17.580000,17.608900,17.620001,17.680000,17.600000,17.600000,17.600000,17.500000,17.490000,17.360001,17.315001,17.230000,17.299999,17.260000,17.250000

In [86]:
tpose = df.drop(index=7).transpose()
tpose.columns = ['VIE', 'API', 'BCRX', 'ARQT', 'GRTS', 'AMTX', 'NKTR', 'EH', 'ADMS']
tpose

,VIE,API,BCRX,ARQT,GRTS,AMTX,NKTR,EH,ADMS
0,52.709999,72.080002,10.7500,33.250000,21.900000,7.9100,22.940001,76.790001,6.6650
1,52.799999,73.846603,10.7376,33.220001,21.500000,7.6500,22.049999,78.156799,6.6500
2,53.000000,72.820000,10.4600,33.799999,21.250000,8.0101,21.450001,79.500000,6.7300
3,53.080002,72.500000,10.4481,34.139999,21.000000,7.9500,21.650000,78.000000,6.8229
4,53.055000,72.589996,10.5000,34.549999,21.398199,7.8400,22.105000,75.959900,6.7158
...,...,...,...,...,...,...,...,...,...
190,52.889999,73.690002,11.9000,NaN,17.200001,7.8200,20.780001,79.800003,NaN
191,52.884998,73.199997,11.9300,NaN,17.150000,7.8200,20.780001,79.980003,NaN
192,52.884998,72.620003,11.9050,NaN,17.160000,7.8150,20.760000,79.760002,NaN
193,52.889999,73.300003,11.8750,NaN,17.180000,NaN,20.735001,80.529999,NaN


Get percent return for each minute of the day, assuming an equal dollar amount held of each stock

In [93]:
arr = df.drop(index=7).to_numpy()

arr

array([[52.70999908, 52.79999924, 53.        , ..., 52.88499832,
        52.88999939, 52.86819839],
       [72.08000183, 73.84660339, 72.81999969, ..., 72.62000275,
        73.30000305, 73.77999878],
       [10.75      , 10.73760033, 10.46000004, ..., 11.90499973,
        11.875     , 11.83500004],
       ...,
       [22.94000053, 22.04999924, 21.45000076, ..., 20.76000023,
        20.73500061, 20.67499924],
       [76.79000092, 78.15679932, 79.5       , ..., 79.76000214,
        80.52999878, 80.27999878],
       [ 6.66499996,  6.6500001 ,  6.73000002, ...,         nan,
                nan,         nan]])

In [94]:
for row in arr:
    first = row[0]
    for i in range(len(row)):
        row[i] = row[i]/first - 1
arr

array([[ 0.        ,  0.00170746,  0.00550182, ...,  0.00332004,
         0.00341492,  0.00300131],
       [ 0.        ,  0.0245089 ,  0.01026634, ...,  0.00749169,
         0.01692565,  0.02358486],
       [ 0.        , -0.00115346, -0.02697674, ...,  0.10744184,
         0.10465116,  0.10093024],
       ...,
       [ 0.        , -0.03879692, -0.06495204, ..., -0.09503053,
        -0.09612031, -0.09873589],
       [ 0.        ,  0.01779917,  0.03529104, ...,  0.03867693,
         0.04870423,  0.0454486 ],
       [ 0.        , -0.00225054,  0.00975245, ...,         nan,
                nan,         nan]])

In [97]:
ardf = pd.DataFrame(arr)
gains = list(ardf.sum(axis=0))
gains

[0.0,
 -0.05022218354864061,
 -0.03160121452192266,
 -0.04129491774432448,
 -0.04188768919335906,
 -0.07509219898772168,
 -0.07240606333060828,
 -0.0804052060448679,
 -0.03667405017448089,
 -0.059499588111388,
 -0.12281965340317502,
 -0.11711416073001701,
 -0.14905965171147895,
 -0.11786842592969582,
 -0.07037266114394247,
 -0.05703895717627783,
 -0.02630820257283295,
 -0.01334470396109333,
 0.017651832472343476,
 0.0171945297913374,
 -0.0056630006563773705,
 -0.001599334193437163,
 -0.022265845926617733,
 -0.06832613404397636,
 -0.07306709835703318,
 -0.10050514757120321,
 -0.12261958964094954,
 -0.12508520721669214,
 -0.11776324058168408,
 -0.14825468748162218,
 -0.1261594632644658,
 -0.10535337361880381,
 -0.15239270963885432,
 -0.2238201606493272,
 -0.23202658511701124,
 -0.23004478512541637,
 -0.185481122702696,
 -0.1857791146513752,
 -0.1728540250591618,
 -0.17137644393361717,
 -0.19390445588145677,
 -0.28395939574126683,
 -0.2991168656707317,
 -0.29839672409530793,
 -0.278800590

In [103]:
print('Good times to sell')
print('Minute after opening: % return')
{i:x * 100 for i, x in enumerate(gains) if x > 0}

Good times to sell
Minute after opening: % return


{18: 1.7651832472343476,
 19: 1.7194529791337398,
 105: 8.670026924541618,
 106: 8.257140482405923,
 107: 3.4094569784644646,
 108: 4.138404009711216,
 118: 0.6493290829930998,
 133: 0.019991328210944292,
 134: 0.3884484772002872,
 146: 0.7910340264229099}